Introducción a las Redes Neuronales para Análisis de Textos, 2024



---
# Ejercicio: Uso de Word Embeddings en un clasificador

En este notebook se explorará una colección de word embeddings preentrenada y se la utilizará para entrenar un clasificador de detección de humor.


---



Descargar una colección de embeddings.

En este ejercicio se pueden utilizar diferentes colecciones de embeddings, que darán distintos resultados. Proveemos tres alternativas, cada una con sus pros y sus contras.

1.   Embeddings del Spanish Billion Word Corpus (SBWCE): tamaño 300, están entrenados con más de mil millones de palabras en español y se cargan con gensim.
2.   Embeddings de Wikipedia en HuggingFace: tamaño 100, están entrenados con el texto de la Wikipedia en español (mucho menos texto) y se cargan con gensim.
3.   Embeddings tipo FastText de TorchNLP: tamaño 300, son otro tipo de vector distinto a word2vec, no podemos utilizar gensim por lo que hay que implementar métodos particulares para los cálculos de similitud.



In [ ]:
# Alternativa 1

# Fromas de bajar los embeddings de SBWC:

# Sitio oficial de la Universidad Nacional de Córdoba
# ! wget https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz

# Mirror en el Owncloud de Fing
# ! wget https://www.fing.edu.uy/owncloud/index.php/s/ryHF9xWwox3NrFe/download/SBW-vectors-300-min5.bin.gz

# Si los enlaces son demasiado lentos: bajarlo una sola vez y copiarlo a google drive
# ! cp /content/drive/MyDrive/share/SBW-vectors-300-min5.bin.gz .

# El archivo viene en un gzip, lo descomprimimos
! gzip -d SBW-vectors-300-min5.bin.gz

# Abrirlos con la biblioteca de embeddings gensim
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format("./SBW-vectors-300-min5.bin", binary=True)
print(wv.vectors.shape)
embeddings_size = wv.vectors.shape[1]

In [ ]:
# Alternativa 2

# Bajar y abrir los embeddings de Wikipedia de HuggingFace

from gensim.models import KeyedVectors
from huggingface_hub import hf_hub_download
wv = KeyedVectors.load_word2vec_format(hf_hub_download(repo_id="Word2vec/wikipedia2vec_eswiki_20180420_100d", filename="eswiki_20180420_100d.txt"))
print(wv.vectors.shape)
embeddings_size = wv.vectors.shape[1]


In [ ]:
# Alternativa 3

# Para utilizar los embeddings tipo FastText de TorchNLP, primero hay que instalar la biblioteca
! pip install pytorch-nlp

import torchnlp
from torchnlp.word_to_vector import FastText

# Cargar los embeddings FastText en español
wv = FastText(language='es')
print(wv.vectors.shape)
embeddings_size = wv.vectors.shape[1]

# Cuidado! Estos embeddings no están cargados con la biblioteca gensim, por lo que no podemos utilizar sus funcionalidades habituales
# Hay que implementar el método de cálculo similaridad (ver sklearn.neighbors.NearestNeighbors)

Probar algunos casos simples de interés.

¿Cómo son los vectores de palabras contenidos en esta colección?

¿Qué otras palabras hay cerca de una palabra objetivo?

In [ ]:
print(wv['perro'])
print(wv.most_similar('perro'))

Probar algunas analogías, utilizando el método most_similar() de gensim:

rey - hombre + mujer ≈ reina

parís - francia + uruguay ≈ montevideo

Buscar por lo menos cuatro ejemplos más de analogías.

In [ ]:
# cálculo de analogías


Probar algunas similitudes entre palabras utilizando el método similarity() de gensim

Considere los pares:

*   perro - gato
*   frío - helado
*   democracia - monarquía
*   frío - caliente

Escribir por lo menos seis pares más.

¿Cuáles deberían estar más cerca según la intuición humana?

¿Se cumple eso en los embeddings?


In [ ]:
# cálculo de similitud


Probar el cálculo de distancia con el método distance() de gensim. ¿Cuál es la relación con el método usado en la parte anterior?

In [ ]:
# cálculo de distancia


Visualizar con dos técnicas de reducción de dimensionalidad: PCA y t-SNE

Probaremos con un conjunto particular de palabras de distintas clases.

Realice más pruebas utilizando otras palabras que le parezcan relevantes.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

words = ['perro','gato','elefante','tiburón','loro','paloma','ballena','democracia','trabajo','economía','política','guerra','aerodinámico','rápido','lento','intenso','furioso','azul','rojo','verde','amarillo','naranja','lunes','martes','domingo','febrero','diciembre','comer','saltar','dormir','volar','salir','entrar']
X = np.array([wv[w] for w in words])

In [ ]:
from sklearn.decomposition import PCA

X_pca = PCA(n_components=2).fit_transform(X)

fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(X_pca[:,0], X_pca[:,1])

for i, w in enumerate(words):
    ax.annotate(w, X_pca[i])

In [ ]:
from sklearn.manifold import TSNE

X_tsne = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=3).fit_transform(X)

fig, ax = plt.subplots(figsize=(10,10))
ax.scatter(X_tsne[:,0], X_tsne[:,1])

for i, w in enumerate(words):
    ax.annotate(w, X_tsne[i])


Obtener y cargar el corpus de detección de humor. Este corpus tiene varias columnas, y fue utilizado en la competencia [HAHA 2021](https://www.fing.edu.uy/inco/grupos/pln/haha/). Las que nos interesarán en este ejercicio son las columnas `text` que contiene el texto del tweet, y la columna `is_humor` que indica si el tweet es un chiste o no.

In [ ]:
! wget https://www.fing.edu.uy/owncloud/index.php/s/PJHnZ3rKe34mqc9/download/haha_2021.zip
! unzip haha_2021.zip

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

train_df = pd.read_csv('./haha_2021_train.csv')
dev_df = pd.read_csv('./haha_2021_dev_gold.csv')
test_df = pd.read_csv('./haha_2021_test_gold.csv')
train_df.sample(5)


Para utilizar los textos del corpus en un clasificador, debemos realizar las siguientes acciones:

1.   Preprocesar los textos (p.e. tokenizar)
2.   Transformarlos a una representación vectorial (p.e. centroide)
3.   Obtener los labels del corpus (que son valores 0 o 1) como array de numpy.


In [ ]:
# código de preprocesamiento

def tokenize(text):
  return ...

train_tokens = [tokenize(t) for t in train_df.loc[:,'text']]
dev_tokens = [tokenize(t) for t in dev_df.loc[:,'text']]
test_tokens = [tokenize(t) for t in test_df.loc[:,'text']]


In [ ]:
# código de vectorización

def get_matrix(tokens):
  # recorrer la lista de tweets
  # cada uno lo pasamos a una representación de vectores (p.e. centroide)
  # juntarlos todos en una matriz de numpy
  return np.array(vectors)

train_v = get_matrix(train_tokens)
dev_v = get_matrix(dev_tokens)
test_v = get_matrix(test_tokens)

print(train_v.shape)
print(dev_v.shape)
print(test_v.shape)


In [ ]:
# código para obtener los labels

train_labels = np.array(train_df.loc[:,'is_humor'])
dev_labels = np.array(dev_df.loc[:,'is_humor'])
test_labels = np.array(test_df.loc[:,'is_humor'])

print("train",np.bincount(train_labels))
print("dev",np.bincount(dev_labels))
print("test",np.bincount(test_labels))


El siguiente código es un script de evaluación muy simple que toma un clasificador, un conjunto de datos y sus labels esperados, y nos devuelve Precisión, Recall, F1 y Accuracy.

Utilizaremos este script de evaluación para comparar todos nuestros resultados.


In [ ]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate(clf,vectors,labels):
  pred = clf.predict(vectors)
  p,r,f,s = precision_recall_fscore_support(labels,pred)
  a = accuracy_score(labels,pred)
  print("P %s, R %s, F %s, A %s" % (p[1],r[1],f[1],a))


Entrenar clasificadores en sklearn intentando encontrar el mejor para los datos de dev. Por ejemplo: LogisticRegression, RandomForestClassifier, SVC.

In [ ]:
clf1 = ... # construyo y entreno el clasificador

evaluate(clf1,dev_v,dev_labels)

In [ ]:
clf2 = ...

evaluate(clf2,dev_v,dev_labels)

Evaluar sobre el conjunto de test la performance del mejor clasificador encontrado

In [ ]:
clf_best = ... # el mejor clasificador que encontré
evaluate(clf_best,test_v,test_labels)